In [ ]:
!pip install ultralytics

In [ ]:
!pip install wandb

In [ ]:
!yolo settings wandb=True

In [ ]:
import wandb
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
wandb_key = user_secrets.get_secret("WANDB_API_KEY")
wandb.login(key=wandb_key)

In [ ]:
import os
import csv
import glob
import shutil
import random
from pathlib import Path
#/kaggle/input/uadamage-demining-competition/train
# Config
BASE_DIR = "/kaggle/input/uadamage-demining-competition/"
BASE_OUTPUT_DIR = "/kaggle/working/"
ANNOTATIONS_DIR = BASE_DIR + "train/annotations"
IMAGES_DIR = BASE_DIR + "train/images"
OUTPUT_LABELS_DIR = BASE_OUTPUT_DIR + "labels"
OUTPUT_IMAGES_DIR = BASE_OUTPUT_DIR + "images"
TRAIN_SPLIT = 0.8

# Make output directories
(Path(OUTPUT_LABELS_DIR) / "train").mkdir(parents=True, exist_ok=True)
(Path(OUTPUT_LABELS_DIR) / "val").mkdir(parents=True, exist_ok=True)
(Path(OUTPUT_IMAGES_DIR) / "train").mkdir(parents=True, exist_ok=True)
(Path(OUTPUT_IMAGES_DIR) / "val").mkdir(parents=True, exist_ok=True)

# Collect all CSV annotation files
csv_files = glob.glob(os.path.join(ANNOTATIONS_DIR, "*.csv"))

# Shuffle and split
random.shuffle(csv_files)
split_idx = int(len(csv_files) * TRAIN_SPLIT)
train_files = csv_files[:split_idx]
val_files = csv_files[split_idx:]

def convert_and_save(csv_path, label_output_dir, image_output_dir):
    with open(csv_path, newline='') as csvfile:
        reader = csv.DictReader(csvfile)
        yolo_lines = []
        image_name = None
        for row in reader:
            image_name = row["image_id"]
            x = float(row["x"])
            y = float(row["y"])
            w = float(row["width"])
            h = float(row["height"])
            img_w = float(row["image_width"])
            img_h = float(row["image_height"])
            cls = row["label"]

            # Convert to YOLO format: normalized center x, y, width, height
            x_center = (x + w / 2) / img_w
            y_center = (y + h / 2) / img_h
            w_norm = w / img_w
            h_norm = h / img_h

            yolo_line = f"{cls} {x_center:.6f} {y_center:.6f} {w_norm:.6f} {h_norm:.6f}"
            yolo_lines.append(yolo_line)

        if image_name:
            # Save label file
            label_path = os.path.join(label_output_dir, f"{image_name}.txt")
            with open(label_path, "w") as f:
                f.write("\n".join(yolo_lines))

            # Copy image file to appropriate folder
            for ext in [".jpg", ".png"]:
                image_path = os.path.join(IMAGES_DIR, f"{image_name}{ext}")
                if os.path.exists(image_path):
                    shutil.copy(image_path, os.path.join(image_output_dir, f"{image_name}{ext}"))
                    break

# Convert files
for file in train_files:
    convert_and_save(file, f"{OUTPUT_LABELS_DIR}/train", f"{OUTPUT_IMAGES_DIR}/train")

for file in val_files:
    convert_and_save(file, f"{OUTPUT_LABELS_DIR}/val", f"{OUTPUT_IMAGES_DIR}/val")



In [ ]:
yaml_content = """
path: /kaggle/working
train: images/train
val: images/val

names:
  0: Explosives
  1: Anti-personnel mine
  2: Anti-vehicle mine
"""

with open("/kaggle/working/dataset.yaml", "w") as f:
    f.write(yaml_content.strip() + "\n")

In [ ]:
from ultralytics import RTDETR

In [ ]:
model = RTDETR("rtdetr-l.pt")
model.info()

In [ ]:
results = model.train(data="/kaggle/working/dataset.yaml", epochs=100,
                      project="rtdetr-l-ultralytics", name="rtdetr-l", batch=8, device=[0,1], save_period = 5)